In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import os

os.system("ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432")
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()
print(session)

In [2]:
#Numero total de artigos no articles.tsv
n_docs = 5982

from snorkel.parser import TSVDocPreprocessor
doc_preprocessor = TSVDocPreprocessor('/home/lzirondi/datasets/Processed/Tweets Anotados.tsv', max_docs=n_docs)

#

from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser(parser=Spacy())

%time corpus_parser.apply(doc_preprocessor, count=n_docs, parallelism=16)

from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())
print()

Clearing existing...
100%|██████████| 5982/5982 [01:17<00:00, 77.11it/s]CPU times: user 2.28 s, sys: 830 ms, total: 3.11 s
Wall time: 1min 20s
Documents: 5982
Sentences: 6035




In [3]:
from snorkel.models import candidate_subclass

causas = candidate_subclass('causas', ['Substância', 'Evento_Adverso'])

#
def subsToList():

    f = open('/home/lzirondi/datasets/Processed/Dicionario_de_Substancias.txt', 'r', encoding='UTF-8')
    s = f.read()
    f.close()
    s = s.splitlines()

    return s

def eventosToList():
    f = open('/home/lzirondi/datasets/Processed/Dicionario_de_Eventos.txt', 'r', encoding='UTF-8')
    s = f.read()
    f.close()
    s = s.splitlines()

    return s

substancias = subsToList()
eventos = eventosToList()



from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import DictionaryMatch

ngrams = Ngrams(n_max=7)
substancia_matcher = DictionaryMatch(d=substancias, longest_match_only=True, ignore_case=True)
eventos_matcher = DictionaryMatch(d=eventos, longest_match_only=True, ignore_case=True)
cand_extractor = CandidateExtractor(causas, [ngrams, ngrams], [substancia_matcher, eventos_matcher])

from snorkel.models import Document
from util import number_of_people

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 8:
            dev_sents.add(s)
        elif i % 10 == 9:
            test_sents.add(s)
        else:
            train_sents.add(s)
                
#

%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i, parallelism = 16)
    print("Number of candidates:", session.query(causas).filter(causas.split == i).count())

0%|          | 0/4828 [00:00<?, ?it/s]CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.78 µs
Clearing existing...
Running UDF...
  0%|          | 0/602 [00:00<?, ?it/s]Number of candidates: 14395
Clearing existing...
Running UDF...
  0%|          | 0/605 [00:00<?, ?it/s]Number of candidates: 1739
Clearing existing...
Running UDF...
100%|██████████| 605/605 [00:15<00:00, 31.85it/s]Number of candidates: 1825



In [4]:
cands = session.query(causas).filter(causas.split == 0).all()


In [5]:
t = cands[40]
a = t.get_contexts()
#print(t)
#print(a)
span1 = a[0]
id = span1.get_stable_id()
print(id)
print(type(id))
s = id.split(':')
print(s)
#0id, 1nada, 2span, 3charinicial, 4final


#print(dir(t))
#print(t.get_contexts)

3929::span:43:50
<class 'str'>
['3929', '', 'span', '43', '50']


In [7]:
#0id, 1nada, 2span, 3charinicial, 4final

query = session.query(Sentence)

tsv = open('candidatos.tsv', 'w', encoding='UTF-8')

#for candidate in session.query(causas).filter(causas.split == 0).all():
#    context = candidate.get_contexts()

candidate = session.query(causas).filter(causas.split == 0).all()

for i in range(13,18):
    context = candidate[i]
    print(context)
    span1 = context[0]
    span2 = context[1]

    span1_text = str(context[0]).split('\"')[1].split('\'')[1] 
    span2_text = str(context[1]).split('\"')[1].split('\'')[1]

    id1 = span1.get_stable_id().split(':')
    id2 = span2.get_stable_id().split(':')

    sentence = query[int(id1[0])]

    asdict = sentence._asdict()

    document = str(asdict['document']).split(' ')[1]
    
    text = asdict['text']
    n1 = int(id1[3]) 
    m1 = int(id1[4])
    cand_1 = text[n1:m1]
    
    n2 = int(id2[3])
    m2 = int(id2[4])
    cand_2 = text[n2:m2]

    text = text.replace(cand_1, cand_1.upper())
    text = text.replace(cand_2, cand_2.upper())
    text = text.replace('\n', '')

    line = document + '\t' + text + '\t' + span1_text + '\t' + span2_text + '\n'
    print(line)

    tsv.write(line)


tsv.close()





causas(Span("b'nimesulida'", sentence=68448, chars=[25,34], words=[6,6]), Span("b'dor de garganta'", sentence=68448, chars=[76,90], words=[16,18]))
2719	 ja tomei dois comprimidoS DE NIMEsulida e essa dor nao passa .	nimesulida	dor de garganta

causas(Span("b'e'", sentence=68448, chars=[47,47], words=[9,9]), Span("b'dor de garganta'", sentence=68448, chars=[76,90], words=[16,18]))
2719	 ja tomei dois comprimidos de nimesulida e essa dor nao passa .	e	dor de garganta

causas(Span("b'e'", sentence=68448, chars=[36,36], words=[7,7]), Span("b'dor de garganta'", sentence=68448, chars=[76,90], words=[16,18]))
2719	 ja tomei dois comprimidos de nimesulida e essa dor nao passa .	e	dor de garganta

causas(Span("b'e'", sentence=68448, chars=[102,102], words=[21,21]), Span("b'dor de garganta'", sentence=68448, chars=[76,90], words=[16,18]))
2719	 ja tomei dois comprimidos de nimesulida e essa dor nao passa .	e	dor de garganta

causas(Span("b'a'", sentence=67061, chars=[50,50], words=[10,10]), Spa

In [6]:
cands = session.query(causas).filter(causas.split == 0).all()
context = cands[0]
spn1 = context[0]
spn2 = context[1]

s = str(spn1).split("\"")
print(s)
s2 = s[1].split('\'')
print(s2)
print(s2[1])

['Span(', "b'dipirona'", ', sentence=70853, chars=[7,14], words=[2,2])']
['b', 'dipirona', '']
dipirona
